# Knative

Knative (ausgesprochen kay-nay-tiv) erweitert Kubernetes um eine Reihe von Middleware-Komponenten, die für die Erstellung moderner, container-basierter Anwendungen unerlässlich sind.

Knative Komponenten konzentrieren sich auf die Lösung alltäglicher, aber schwieriger Aufgaben wie:

* Bereitstellen eines Containers
* Routing und Verwaltung des Datenverkehrs mit blue / green Bereitstellung
* Automatische Skalierung des Workloads
* Bindung laufender Dienste an mehrere Ökosysteme

***
## [Autoscaling](https://knative.dev/docs/serving/configuring-autoscaling/)

Knative unterstützt folgende Arten von Autoscaling:
* concurrency  
* requests-per-seconds (rps)

### Installation

In [ ]:
%%bash
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.7.2/serving-crds.yaml
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.7.2/serving-core.yaml
kubectl apply -l knative.dev/crd-install=true -f https://github.com/knative/net-istio/releases/download/knative-v1.7.0/istio.yaml
kubectl apply -f https://github.com/knative/net-istio/releases/download/knative-v1.7.0/istio.yaml
kubectl apply -f https://github.com/knative/net-istio/releases/download/knative-v1.7.0/net-istio.yaml
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.7.2/serving-hpa.yaml

Installation überprüfen. Es müssen 9 Container gestartet sein.

In [ ]:
%%bash
kubectl get pods -n knative-serving
kubectl --namespace istio-system get service istio-ingressgateway

***
### Beispiel Autoscaling 

Wir haben einen Service welcher immer wieder Spitzenbelastungen aufweist, z.B. Black Friday etc.

Dieser soll automatisch hochskaliert werden, wenn zuwenig Pods vorhanden sind.

In [ ]:
! kubectl create namespace autoscaling

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: serving.knative.dev/v1 # Current version of Knative
kind: Service
metadata:
  name: helloworld-go # The name of the app
  namespace: autoscaling # The namespace the app will use
spec:
  template:
    metadata:
      annotations:
        autoscaling.knative.dev/metric: "rps"  
        autoscaling.knative.dev/target: "150"        
    spec:
      containers:
        - image: gcr.io/knative-samples/helloworld-go # The URL to the image of the app
          env:
            - name: TARGET # The environment variable printed out by the sample app
              value: "ich werde automatisch Skaliert!"
%EOF%

Schauen wir uns nun die erstellten Ressourcen an.

Der eigentliche Service (Pods) wird am Anfang gestartet und nach einer bestimmen Leerlaufzeit beendet. Das ReplicaSet setzt die Anzahl Pods auf 0.

In [ ]:
! kubectl get all --namespace autoscaling

Nun erzeugen wir Last auf dem Services und schauen an wie die Pods skaliert werden.

In [ ]:
%%bash
for x in {1..1000}
do
curl -s -H "Host: helloworld-go.autoscaling.example.com" istio-ingressgateway.istio-system >/dev/null &
done
kubectl get pods --namespace autoscaling

- - -
Aufräumen, kann auch Verwendet werden wenn die Ressoucen auf Status Ready=Unknown verbleiben.

In [ ]:
! kubectl delete namespace autoscaling